In [1]:
import os
import pandas as pd

In [24]:
import requests
import datetime
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import DatasetDriftMetric, DatasetMissingValuesMetric, ColumnSummaryMetric, ColumnDriftMetric

import joblib

In [4]:
os.getcwd()

'/Users/tunahankilic/Desktop/NBAGamePrediction/notebooks'

In [5]:
os.chdir('../')

In [6]:
from src import config

In [7]:
num_features = config.BEST_COLUMNS

In [8]:
reference_data = pd.read_parquet(os.path.join(config.OUTPUT_PATH, "nba-2022-season-predictions-reference.parquet"))
raw_data = pd.read_parquet(os.path.join(config.OUTPUT_PATH, "test.parquet"))

In [9]:
with open('models/rf_nba.bin', 'rb') as f_in:
	model = joblib.load(f_in)

In [10]:
current_data = raw_data.copy()
current_data['game_prediction'] = model.predict(current_data[num_features])

In [11]:
reference_data.head()

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,pace,ft/fga,team,total,home,mp_opp,fg_opp,...,fg_opp_10_nopp,fga_opp_10_nopp,fg%_opp_10_nopp,3p_opp_10_nopp,3pa_opp_10_nopp,3p%_opp_10_nopp,ft_opp_10_nopp,fta_opp_10_nopp,ft%_opp_10_nopp,orb_opp_10_nopp,drb_opp_10_nopp,trb_opp_10_nopp,ast_opp_10_nopp,stl_opp_10_nopp,blk_opp_10_nopp,tov_opp_10_nopp,pf_opp_10_nopp,pts_opp_10_nopp,ts%_opp_10_nopp,efg%_opp_10_nopp,3par_opp_10_nopp,ftr_opp_10_nopp,orb%_opp_10_nopp,drb%_opp_10_nopp,trb%_opp_10_nopp,ast%_opp_10_nopp,stl%_opp_10_nopp,blk%_opp_10_nopp,tov%_opp_10_nopp,usg%_opp_10_nopp,ortg_opp_10_nopp,drtg_opp_10_nopp,pace_opp_10_nopp,ft/fga_opp_10_nopp,total_opp_10_nopp,home_opp_10_nopp,days_to_next_game_10_nopp,team_opp_next_nopp,team_nopp,game_prediction
0,240.0,31.0,88.0,0.352,11.0,44.0,0.250,12.0,16.0,0.750,11.0,31.0,42.0,23.0,6.0,3.0,17.0,18.0,85.0,0.447,0.415,0.500,0.182,21.6,77.5,46.2,74.2,6.1,7.3,15.2,100.0,87.1,129.1,97.6,0.136,NOP,85,0,240.0,47.0,...,41.0,82.9,0.4942,12.7,34.8,0.3646,19.1,23.6,0.8220,9.0,33.9,42.9,23.2,7.1,5.0,10.5,19.7,113.8,0.6109,0.5707,0.4207,0.2858,23.17,77.23,51.79,56.34,7.60,10.07,10.11,100.0,122.12,117.33,93.34,0.2312,113.8,0.5,2.6,NOP,DAL,0
1,240.0,38.0,83.0,0.458,10.0,30.0,0.333,16.0,19.0,0.842,11.0,28.0,39.0,22.0,4.0,3.0,17.0,13.0,102.0,0.558,0.518,0.361,0.229,29.7,59.6,46.4,57.9,4.4,4.2,15.7,100.0,113.2,112.1,90.1,0.193,CLE,102,0,240.0,42.0,...,38.0,88.3,0.4314,12.7,35.9,0.3634,15.8,18.8,0.8238,8.7,37.1,45.8,23.6,4.5,4.9,10.5,17.8,104.5,0.5424,0.5036,0.4067,0.2124,18.68,81.69,50.34,62.19,4.70,8.75,9.81,100.0,109.61,107.14,93.51,0.1788,104.5,0.5,2.6,CLE,NYK,1
2,240.0,35.0,89.0,0.393,12.0,41.0,0.293,7.0,10.0,0.700,7.0,36.0,43.0,23.0,9.0,12.0,18.0,13.0,89.0,0.476,0.461,0.461,0.112,14.0,87.8,47.3,65.7,8.7,16.9,16.2,100.0,86.2,98.7,103.3,0.079,ORL,89,1,240.0,40.0,...,42.4,88.8,0.4789,12.9,34.3,0.3741,18.7,22.8,0.8287,10.0,30.9,40.9,20.8,7.4,4.1,10.0,21.6,116.4,0.5911,0.5527,0.3904,0.2649,22.65,78.23,49.21,49.47,7.70,10.39,9.20,100.0,121.39,121.33,96.07,0.2174,116.4,0.4,2.6,ORL,UTA,0
3,240.0,42.0,99.0,0.424,12.0,27.0,0.444,5.0,9.0,0.556,19.0,26.0,45.0,20.0,10.0,8.0,9.0,19.0,101.0,0.490,0.485,0.273,0.091,40.4,70.3,53.6,47.6,11.1,15.1,8.0,100.0,112.1,113.2,90.1,0.051,TOR,101,1,240.0,38.0,...,39.4,88.5,0.4457,10.9,35.3,0.3036,16.9,22.3,0.7188,10.8,34.5,45.3,19.2,6.3,5.5,12.6,18.5,106.6,0.5425,0.5078,0.4003,0.2537,23.96,82.07,51.99,48.63,6.53,10.90,11.37,100.0,110.27,117.54,96.84,0.1932,106.6,0.5,2.5,TOR,BRK,0
4,240.0,43.0,88.0,0.489,14.0,42.0,0.333,6.0,10.0,0.600,7.0,33.0,40.0,28.0,8.0,2.0,11.0,18.0,106.0,0.574,0.568,0.477,0.114,16.3,84.6,48.8,65.1,8.5,4.7,10.6,100.0,112.0,116.2,94.6,0.068,IND,106,0,240.0,40.0,...,40.3,86.5,0.4669,10.3,31.3,0.3208,18.2,24.4,0.7589,10.6,35.3,45.9,24.1,7.4,6.4,15.7,15.6,109.1,0.5625,0.5263,0.3620,0.2836,25.04,80.12,53.09,59.83,7.43,11.68,13.91,100.0,109.42,110.46,99.69,0.2116,109.1,0.6,1.8,IND,SAC,0


In [34]:
report = Report(metrics=[
    DatasetDriftMetric(),
    DatasetMissingValuesMetric(),
    ColumnSummaryMetric(column_name="fg_opp_10"),
    ColumnSummaryMetric(column_name="fg_10_nopp"),
    ColumnDriftMetric(column_name="fg_opp_10", stattest="wasserstein"),
    ColumnDriftMetric(column_name="fg_10_nopp", stattest="wasserstein")
]
)

In [14]:
reference_data['game_prediction'] = reference_data['game_prediction'].astype(object)
current_data['game_prediction'] = current_data['game_prediction'].astype(object)

In [35]:
column_mapping = ColumnMapping(
    numerical_features=num_features,
    target=None,
    prediction='game_prediction'
)

In [37]:
report.run(reference_data=reference_data, current_data=current_data, column_mapping=column_mapping)

In [38]:
result = report.as_dict()

In [39]:
result

{'metrics': [{'metric': 'DatasetDriftMetric',
   'result': {'drift_share': 0.5,
    'number_of_columns': 38,
    'number_of_drifted_columns': 21,
    'share_of_drifted_columns': 0.5526315789473685,
    'dataset_drift': True}},
  {'metric': 'DatasetMissingValuesMetric',
   'result': {'current': {'different_missing_values': {'': 0,
      -inf: 0,
      None: 0,
      inf: 0},
     'number_of_different_missing_values': 0,
     'different_missing_values_by_column': {'mp': {'': 0,
       -inf: 0,
       None: 0,
       inf: 0},
      'fg': {'': 0, -inf: 0, None: 0, inf: 0},
      'fga': {'': 0, -inf: 0, None: 0, inf: 0},
      'fg%': {'': 0, -inf: 0, None: 0, inf: 0},
      '3p': {'': 0, -inf: 0, None: 0, inf: 0},
      '3pa': {'': 0, -inf: 0, None: 0, inf: 0},
      '3p%': {'': 0, -inf: 0, None: 0, inf: 0},
      'ft': {'': 0, -inf: 0, None: 0, inf: 0},
      'fta': {'': 0, -inf: 0, None: 0, inf: 0},
      'ft%': {'': 0, -inf: 0, None: 0, inf: 0},
      'orb': {'': 0, -inf: 0, None: 0, inf

In [40]:
result['metrics'][0]['result']['number_of_drifted_columns']

21

In [41]:
result['metrics'][1]['result']['current']['share_of_missing_values']

0.0

In [49]:
result['metrics'][2]['result']['current_characteristics']['p50']

41.8

In [48]:
result['metrics'][3]['result']['current_characteristics']['p50']

42.0

In [45]:
result['metrics'][4]['result']['drift_score']

0.6020200082652506

In [51]:
result['metrics'][5]['result']['drift_score']

0.6040030968801844